Dans ce post, nous allons implémenter l'algorithme du "radoteur", un algorithme inventé par Roland Moreno.

Ce post se base sur l'explication de l'algorithme trouvée [ici](ftp://ftp-public.4d.fr/Notes_Techniques/NT/2001/200107-23-Radoteur.pdf).

# Description de l'algorithme


Pour fonctionner, l'algorithme a besoin de données d'entrée, qui est une série de mots. Dans notre post nous utiliserons la chaîne de caractères suivante :

> SOIENT CINQUANTE MOTS D’ANGLAIS PRIS AU HASARD, SHANNON POSTULE L’EXISTENCE D’UNE LOI DE COMPOSITION DE CE SOUS-ENSEMBLE DE LA LANGUE, NON NEUTRE, ENTIÈREMENT DÉTERMINÉ, ET VA S’ATTACHER À DEMONTRER LA RIGUEUR DE CETTE LOI.

Ensuite, l'algorithme procède de la manière suivante :

- initialisation : on tire un mot de la suite au hasard et on garde la première lettre, ce qui définit en même temps une position donnée dans la chaîne
- construction de caractères successifs 
    - à partir de la position courante, on avance dans la chaîne jusqu'à rencontrer à nouveau la lettre en mémoire
    - la prochaine lettre à garder est la lettre qui vient après la lettre trouvée
    - on reprend la recherche avec la nouvelle lettre
- l'algorithme prend fin quand le dernier caractère est un espace, une virgule...

# Implémentation 

On définit tout d'abord la chaîne de caractères avec laquelle on travaille :

In [1]:
input_string = "SOIENT CINQUANTE MOTS D’ANGLAIS PRIS AU HASARD, SHANNON POSTULE L’EXISTENCE D’UNE LOI DE COMPOSITION DE CE SOUS-ENSEMBLE DE LA LANGUE, NON NEUTRE, ENTIÈREMENT DÉTERMINÉ, ET VA S’ATTACHER À DEMONTRER LA RIGUEUR DE CETTE LOI."

Ensuite, on choisit l'un des mots présents pour démarrer notre algorithme.

In [2]:
words = input_string.split(' ')

In [3]:
from random import randint

In [4]:
words[randint(0, len(words) - 1)]

'LA'

On peut résumer ces étapes sous la forme d'une fonction :

In [5]:
def init_chain(input_string):
    """Retourne l'un de mots de la chaîne, choisi au hasard."""
    words = input_string.split(' ')
    return words[randint(0, len(words) - 1)]

On vérifie le fonctionnement de la fonction :

In [6]:
init_chain(input_string)

'LOI.'

In [7]:
init_chain(input_string)

'SOIENT'

On peut maintenant implémenter la recherche du prochain caractère. Tout d'abord, il s'agit de retrouver la position du mot de départ et de démarrer la recherche de la prochaine occurence de la lettre courante à partir de cette position.

In [8]:
word = 'ENTIÈREMENT'
position = input_string.index(word)
position

147

On peut réagencer la chaîne de caractère pour rechercher la prochaine occurence. La nouvelle chaîne démarre à la fin de l'occurence du mot courant et finit par le bout du départ.

In [9]:
new_input_string = input_string[position + len(word):] + input_string[:position + len(word)]
new_input_string

' DÉTERMINÉ, ET VA S’ATTACHER À DEMONTRER LA RIGUEUR DE CETTE LOI.SOIENT CINQUANTE MOTS D’ANGLAIS PRIS AU HASARD, SHANNON POSTULE L’EXISTENCE D’UNE LOI DE COMPOSITION DE CE SOUS-ENSEMBLE DE LA LANGUE, NON NEUTRE, ENTIÈREMENT'

Enfin, on peut trouver la première occurence de la lettre courante :

In [10]:
new_input_string.index(word[0])

4

Enfin, on peut sélectionner la prochaine lettre :

In [11]:
new_input_string[5]

'R'

Résumons ceci sous forme d'une fonction :

In [12]:
def build_chain(input_string, word):
    """Construit une chaîne selon l'algorithme du radoteur."""
    chain = word[0]
    position = input_string.index(word)
    new_input_string = input_string[position + len(word):] + input_string[:position + len(word)]
    while chain[-1] not in (' ', ',', '.') and len(chain) < 30:
        index = new_input_string.index(chain[-1])
        chain += new_input_string[(index + 1) % len(new_input_string)]
        try:
            position = new_input_string.index(' ', index + 1)
        except ValueError:
            position = 0
        new_input_string = new_input_string[position:] + new_input_string[:position]
    return chain 

Testons :

In [13]:
build_chain(input_string, word)

'ER '

On peut maintenant générer pleins de mots de cette manière : 

In [14]:
set([build_chain(input_string, init_chain(input_string)) for i in range(500)]) 

{'ASHEMONCOUEUE ',
 'CE ',
 'CENGUANNCOUEUE ',
 'CIS ',
 'COUEUE ',
 'D,',
 'DE ',
 'DET ',
 'DEUANNCOUEUE ',
 'ER ',
 'EREUANNCOUEUE ',
 'HA ',
 'LANGUTINTT ',
 'LAUL’US’AUL’US’AUL’US’AUL’US’A',
 'LE ',
 'LEXI ',
 'LOMBLANONTE ',
 'MPRD’A ',
 'NCOUEUE ',
 'NTETTRI.',
 'POITRER ',
 'POUEUE ',
 'RISIÈRMOINGUTINTT ',
 'S ',
 'SOTUN ',
 'STE ',
 'S’AUL’US’AUL’US’AUL’US’AUL’US’',
 'VATRI.',
 'À '}

# Conclusion 

Nous avons implémenté l'algorithme du radoteur. Les résultats sont intéressants, mais pas forcéments tous valides. Je pense que ceci explique pourquoi cet algorithme ne peut fournir que des candidats qu'il faut ensuite trier manuellement pour retenir ceux qui sont pertinents. Dans le cas présent, les mots intéressants que je garderais sont : 

- poitrer
- lexi
- coueue
- lomblanonte